<h1>Exercise 39</h1>
<p>Write a query to determine phone #'s that satisfy these conditions:</p>
<ul>
<li>The #'s have both incoming and outgoing calls</li>
<li>The sum of duration of outgoing calls should be greater than sum of duration of incoming calls:</li>
</ul>
<strong>Expected outcome:</strong>
<br>
<img src="Day-39-Expected_Output.png" width="150">

In [2]:
import os
import sqlite3
import pandas as pd

db_name="exercise39.db"
conn=sqlite3.connect(db_name)
cursor=conn.cursor()

cursor.execute("""CREATE TABLE IF NOT EXISTS call_details(call_type VARCHAR(10), call_number VARCHAR(12), call_duration INT)""")
conn.commit()

call_data=[
    ('OUT','181868',13),
    ('OUT','2159010',8),
    ('OUT','2159010',178),
    ('SMS','4153810',1),
    ('OUT','2159010',152),
    ('OUT','9140152',18),
    ('SMS','4162672',1),
    ('SMS','9168204',1),
    ('OUT','9168204',576),
    ('INC','2159010',5),
    ('INC','2159010',4),
    ('SMS','2159010',1),
    ('SMS','4535614',1),
    ('OUT','181868',20),
    ('INC','181868',54),
    ('INC','218748',20),
    ('INC','2159010',9),
    ('INC','197432',66),
    ('SMS','2159010',1),
    ('SMS','4535614',1)
]
cursor.executemany("INSERT INTO call_details VALUES(?,?,?)",call_data)

querytoexec="SELECT * FROM call_details"
df=pd.read_sql(querytoexec,conn)
print(df)

   call_type call_number  call_duration
0        OUT      181868             13
1        OUT     2159010              8
2        OUT     2159010            178
3        SMS     4153810              1
4        OUT     2159010            152
5        OUT     9140152             18
6        SMS     4162672              1
7        SMS     9168204              1
8        OUT     9168204            576
9        INC     2159010              5
10       INC     2159010              4
11       SMS     2159010              1
12       SMS     4535614              1
13       OUT      181868             20
14       INC      181868             54
15       INC      218748             20
16       INC     2159010              9
17       INC      197432             66
18       SMS     2159010              1
19       SMS     4535614              1


In [3]:
first_sol = "WITH cte AS (SELECT call_number, SUM(CASE WHEN call_type = 'OUT' THEN call_duration ELSE NULL END) AS out_duration, SUM(CASE WHEN call_type = 'INC' THEN call_duration ELSE NULL END) AS inc_duration FROM call_details GROUP BY call_number) SELECT call_number FROM cte WHERE out_duration IS NOT NULL AND inc_duration IS NOT NULL AND out_duration > inc_duration"
df=pd.read_sql(first_sol,conn)
print(df)

  call_number
0     2159010


In [4]:
second_sol = "SELECT call_number FROM call_details GROUP BY call_number HAVING SUM(CASE WHEN call_type='OUT' THEN call_duration ELSE NULL END) > 0 AND SUM(CASE WHEN call_type='INC' THEN call_duration ELSE NULL END) > 0 AND SUM(CASE WHEN call_type='OUT' THEN call_duration ELSE NULL END) > SUM(CASE WHEN call_type='INC' THEN call_duration ELSE NULL END)"
df=pd.read_sql(second_sol,conn)
print(df)

  call_number
0     2159010


In [5]:
third_sol = "WITH cte_out AS (SELECT call_number, SUM(call_duration) AS duration FROM call_details WHERE call_type='OUT' GROUP BY call_number), cte_in AS (SELECT call_number, SUM(call_duration) AS duration FROM call_details WHERE call_type='INC' GROUP BY call_number) SELECT cte_out.call_number FROM cte_out INNER JOIN cte_in ON cte_out.call_number=cte_in.call_number WHERE cte_out.duration > cte_in.duration"
df=pd.read_sql(third_sol,conn)
print(df)
conn.close()

  call_number
0     2159010
